In [2]:
import os
from datetime import datetime
import shutil
import yaml
from tqdm import tqdm
import torch
import numpy as np
import sys
%matplotlib widget
import matplotlib as mpl
from matplotlib import pyplot as plt 


import scipy.io as sio
import logging
from transformers import GPT2Model, GPT2Tokenizer, GPT2Config
from collections import namedtuple, defaultdict
import transformers as tr
from transformers import pipeline
import pandas as pd 
from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise 
from itertools import cycle, islice
import scipy as sc
import scipy.io as sio
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm
import bcolz
import pickle

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
save_path="/Users/eghbalhosseini/MyData/semantic_proj/"
analysis_path="/Users/eghbalhosseini/MyData/semantic_proj/analysis/"
analysis_type='compare_NN_empedding_with_sub_rating'
model_type="gpt2-xl"
LAYER_COUNT = 48
FEATURE_COUNT = 1024
access_rights = 0o755
try:
    os.mkdir(save_path,access_rights)
except OSError:
    print('Creation of %s failed\n'%save_path)
model_path=os.path.join(save_path,model_type)
try:
    os.mkdir(model_path,access_rights)
except OSError:
    print('Creation of %s failed\n'%model_path)
    
try:
    os.mkdir(analysis_path,access_rights)
except OSError:
    print('Creation of %s failed\n'%analysis_path)
model_path=os.path.join(analysis_path,analysis_type)
try:
    os.mkdir(model_path,access_rights)
except OSError:
    print('Creation of %s failed\n'%model_path)

Creation of /Users/eghbalhosseini/MyData/semantic_proj/ failed

Creation of /Users/eghbalhosseini/MyData/semantic_proj/gpt2-xl failed

Creation of /Users/eghbalhosseini/MyData/semantic_proj/analysis/ failed

Creation of /Users/eghbalhosseini/MyData/semantic_proj/analysis/compare_NN_empedding_with_sub_rating failed



In [4]:
def match_tokenized_to_untokenized_gpt2(tokenized_sent, untokenized_sent):
    mapping = defaultdict(list)
    untokenized_sent_index = 0
    tokenized_sent_index = 0
    while (untokenized_sent_index < len(untokenized_sent) and
        tokenized_sent_index < len(tokenized_sent)):
      while (tokenized_sent_index + 1 < len(tokenized_sent) and
          not(tokenized_sent[tokenized_sent_index + 1].startswith("Ġ"))):
        mapping[untokenized_sent_index].append(tokenized_sent_index)
        tokenized_sent_index += 1
      mapping[untokenized_sent_index].append(tokenized_sent_index)
      untokenized_sent_index += 1
      tokenized_sent_index += 1
    return mapping

In [6]:
# configure the gpt2 model 
config=GPT2Config.from_pretrained(model_type)
config.output_hidden_states=True
# make model from config
model=GPT2Model(config)
model.from_pretrained(model_type)
device = torch.device("cpu")
model.to(device)
# 
tokenizer = GPT2Tokenizer.from_pretrained(model_type)

domains : animal, cities ... 

dimensions : size, weight, ... 

In [7]:
SEMANTIC_DAT_DIR='data_from_semantic_paper'
SEMANTIC_DAT_FILE='GabeDimensions.xlsx'
SUBJECT_RATING_FILE='rating_as_struct.mat'
Dimensions = pd.read_excel(os.path.join(save_path,SEMANTIC_DAT_DIR,SEMANTIC_DAT_FILE),sheet_name='Dimensions')
Domains = pd.read_excel(os.path.join(save_path,SEMANTIC_DAT_DIR,SEMANTIC_DAT_FILE),sheet_name='Domains top 25',skiprows=1)
DIMENSIONSxDOMAINS=pd.read_excel(os.path.join(save_path,SEMANTIC_DAT_DIR,SEMANTIC_DAT_FILE),sheet_name='DIMENSIONSxDOMAINS',skiprows=0)
subject_ratings=sio.loadmat(os.path.join(save_path,SEMANTIC_DAT_DIR,SUBJECT_RATING_FILE))
# 
DIMENSIONSxDOMAINS=DIMENSIONSxDOMAINS.drop(DIMENSIONSxDOMAINS.index[18:],axis=0)
DIMENSIONSxDOMAINS=DIMENSIONSxDOMAINS.drop(['Unnamed: 21','Domain Usefulness','Number of Greens'],axis=1)
# 

In [8]:
rating_fields=[]
for idx,x in enumerate(subject_ratings['rating_as_struct']['rating_fields'][0,0].transpose()):
    #print((x[0][0]))
    rating_fields.append((x[0][0]))
words=[]
for idx,x in enumerate(subject_ratings['rating_as_struct']['words'][0,0].transpose()):
    word=[str(y[0].squeeze()) for y in x[0]]
    words.append(word)
sub_score=[]
for idx,x in enumerate(subject_ratings['rating_as_struct']['sub_ratings'][0,0].transpose()):
    sub_score.append(x[0])


In [9]:
for idx,domain in tqdm(enumerate(rating_fields)):
    # 1st: get the layerwise representation for each layer of the network for each domain 
    lineList=words[idx]
    representation_arr = np.zeros((len(lineList),2), dtype=np.object)
    for index, line in enumerate(lineList):
        untokenized_word = line.strip().split()
        tokenized_word = tokenizer.tokenize(line,add_prefix_space=True)
        untok_tok_mapping = match_tokenized_to_untokenized_gpt2(tokenized_word, untokenized_word)
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_word)
        segment_ids = [1 for x in tokenized_word]
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segment_ids])
        with torch.no_grad():
            encoded_layers = model(tokens_tensor)
            last_hidden_state=encoded_layers[0]
            hidden_output=encoded_layers[2]
            layer_wise_feature=[torch.stack([torch.mean(F[0,untok_tok_mapping[i][0]:untok_tok_mapping[i][-1]+1,:], dim=0) for i in range(len(untokenized_word))], dim=0) for F in hidden_output]
            layer_wise_reperensetation= [R.view(1,*R.size()) for R in layer_wise_feature ]
            rep_arr=[np.squeeze(x.numpy()) for x in layer_wise_reperensetation]
            representation_arr[index,0]=' '.join(untokenized_word)
            representation_arr[index,1]=rep_arr
    # 2nd: get get layerwise dimensions for the corresponding domain
    dimension=domain.split('_')[1]
    a=np.argwhere([x.lower()==dimension for x in Dimensions['Dimension']])
    b=Dimensions.iloc[a[0]].iloc[0][1:]
    lineList=b
    dimensions_arr = np.zeros((len(lineList),2), dtype=np.object)
    for index, line in enumerate(lineList):
        untokenized_word = line.strip().split()
        tokenized_word = tokenizer.tokenize(line,add_prefix_space=True)
        untok_tok_mapping = match_tokenized_to_untokenized_gpt2(tokenized_word, untokenized_word)
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_word)
        segment_ids = [1 for x in tokenized_word]
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segment_ids])
        with torch.no_grad():
            encoded_layers = model(tokens_tensor)
            last_hidden_state=encoded_layers[0]
            hidden_output=encoded_layers[2]
            layer_wise_feature=[torch.stack([torch.mean(F[0,untok_tok_mapping[i][0]:untok_tok_mapping[i][-1]+1,:], dim=0) for i in range(len(untokenized_word))], dim=0) for F in hidden_output]
            layer_wise_reperensetation= [R.view(1,*R.size()) for R in layer_wise_feature ]
            rep_arr=[np.squeeze(x.numpy()) for x in layer_wise_reperensetation]
            dimensions_arr[index,0]=' '.join(untokenized_word)
            dimensions_arr[index,1]=rep_arr
        
        # 3rd : project the data onto the representation axis
    dim_positive=dimensions_arr[0:3]
    dim_negative=dimensions_arr[3:]
    sem_vec=[[np.stack(y[1])-np.stack(x[1]) for y in dim_negative] for x in dim_positive]
    sem_vec_1=[np.stack(x) for x in sem_vec]
    sem_vec_2=np.concatenate(sem_vec_1,axis=0)
    sem_vec_ave=np.mean(sem_vec_2,axis=0)
        # 4th : get the word projection onto the dimension axis across layers
    NN_resp_projection=[]
    NN_resp_names=[]
    for i, x in enumerate(representation_arr):
        NN_response=np.stack(x[1]);
        a=np.dot(sem_vec_ave,NN_response.transpose())
        NN_resp_projection.append(np.diag(a))
        NN_resp_names.append(x[0])
        
    #5th get subject data 
    score=sub_score[idx]
    score_ave=score.mean(1)
    # 6th generate figures 
    NN_rep_proj_mat=np.stack(NN_resp_projection).transpose()
    plt.figure(figsize=[20,25])
    x=np.asarray(score_ave).reshape(-1, 1)
    for i, y in enumerate(NN_rep_proj_mat):
        regr = linear_model.LinearRegression()
        y=y.reshape(-1, 1)
        regr.fit(x, y)
        y_pred = regr.predict(x)
        plt.subplot(10,5,i+1)
        plt.scatter(x,y,s=5,c=[0,0,0])
        plt.plot(x, y_pred, color='blue', linewidth=3)
        a=r2_score(y,y_pred)
        s=f"{a:.3f}"
        plt.title('R squared: '+s)
    plt.savefig(model_path+'/'+model_type+'_vs_subject_'+domain+'.pdf', format='pdf',bbox_inches='tight')
    plt.close()

0it [00:00, ?it/s]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

1it [00:16, 16.19s/it]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

2it [00:32, 16.30s/it]

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

15it [04:21, 18.61s/it]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

16it [04:40, 18.71s/it]

IndexError: index 0 is out of bounds for axis 0 with size 0